#**Fine-tune an Open-source LLM on a custom PDF**#

###Installing dependencies###

In [ ]:
!pip install --quiet --upgrade "transformers>=4.34.0" "huggingface_hub>=0.18.0" accelerate bitsandbytes peft datasets sentencepiece pymupdf

In [ ]:
!pip install PyPDF2

In [ ]:
!pip install transformers

###Upload and Extract PDF###

In [ ]:
from google.colab import files
print("Upload your Machine Learning Yearning PDF now (drag & drop)...")
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]
print("Uploaded file:", pdf_path)

In [ ]:
from transformers import pipeline
import PyPDF2

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device= 0
)

# Function to read PDF and extract text per page
def read_pdf(file_path):
    with open(file_path, "rb") as f:
        pdf = PyPDF2.PdfReader(f)
        text = [page.extract_text() for page in pdf.pages]
    return text

# Function to split text into safe chunks under model token limit
def chunk_text(text, max_chars=1800):
    chunks = []
    current_chunk = ""
    for paragraph in text.split("\n"):
        if len(current_chunk) + len(paragraph) <= max_chars:
            current_chunk += " " + paragraph
        else:
            chunks.append(current_chunk.strip())
            current_chunk = paragraph
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

# Summarize per chapter/page
def summarize_pdf(pdf_path):
    pages = read_pdf(pdf_path)
    summaries = []
    for i, page_text in enumerate(pages):
        if not page_text or len(page_text.strip()) == 0:
            continue
        chunks = chunk_text(page_text)
        piece_summaries = []
        for chunk in chunks:
            try:
                s = summarizer(
                    chunk,
                    max_length=70,
                    min_length=30,
                    do_sample=False,
                    truncation=True  # ensures safety
                )[0]["summary_text"]
                piece_summaries.append(s)
            except Exception as e:
                print(f"Error summarizing chunk on page {i+1}: {e}")
        chapter_summary = " ".join(piece_summaries)
        summaries.append(f"Chapter {i+1} Summary:\n{chapter_summary}\n")
    return summaries

# Run summarization
pdf_path = "/content/Machine Learning Yearning.pdf"
chapter_summaries = summarize_pdf(pdf_path)

# Save output
with open("ml_yearning_summaries.txt", "w") as f:
    for summary in chapter_summaries:
        f.write(summary + "\n")

print("Summarization complete. Saved to ml_yearning_summaries.txt")


In [ ]:
import re, json
import fitz  # PyMuPDF

def extract_pdf_text(path):
    doc = fitz.open(path)
    pages = []
    for i in range(len(doc)):
        text = doc[i].get_text("text") or ""
        pages.append(text)
    return "\n\n".join(pages)

raw_text = extract_pdf_text(pdf_path)
# Normalize newlines
raw_text = re.sub(r"\r\n", "\n", raw_text)
raw_text = re.sub(r"\n{3,}", "\n\n", raw_text)

chapter_regex = r'(?i)(?=(^Chapter\s+\d+\b))'
parts = re.split(chapter_regex, raw_text, flags=re.MULTILINE)

chapters = []
i = 0

while i < len(parts):
    part = parts[i].strip()
    if re.match(r'(?i)^Chapter\s+\d+\b', part):
        title = part
        content = parts[i+1] if i+1 < len(parts) else ""
        if content.strip():
            chapters.append({"title": title, "content": content.strip()})
        i += 2
    else:
        # treat any leading text (preface/introduction) as Preface if non-empty
        if part:
            chapters.insert(0, {"title": "Preface", "content": part})
        i += 1

if len(chapters) == 0:
    chapters = [{"title":"Document", "content": raw_text}]

print(f"Detected {len(chapters)} chapter(s). Example title: {chapters[0]['title']}")

In [ ]:
out_jsonl = "mly_finetune.jsonl"
with open(out_jsonl, "w", encoding="utf-8") as f:
    for ch in chapters:
        ex = {
            "instruction": f"Summarize the following chapter ({ch['title']}) from the textbook in detail, covering key concepts and important points.",
            "input": ch["content"],
            "output": ch.get("summary", "")
        }
        f.write(json.dumps(ex, ensure_ascii=False) + "\n")
print("Saved", out_jsonl, "with", len(chapters), "examples.")

###Load Dataset###

In [ ]:
from datasets import load_dataset
ds = load_dataset("json", data_files=out_jsonl, split="train")
print("Dataset loaded, examples:", len(ds))

###Tokenizer###

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

def load_base_model_or_fallback(model_id):
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=False)
        if tokenizer.pad_token is None:
            tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
        )
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            quantization_config=bnb_config,
            trust_remote_code=False,
        )
        return tokenizer, model
    except Exception as e:
        print("Failed to load", model_id, "->", str(e))
        return None, None

primary_model = "tiiuae/falcon-7b-instruct"
tokenizer, model = load_base_model_or_fallback(primary_model)

if model is None:

    fallback_model = "openlm-research/open_llama_3b_v2"
    print("Falling back to", fallback_model)
    tokenizer, model = load_base_model_or_fallback(fallback_model)
    if model is None:
        raise RuntimeError("Couldn't load primary or fallback model. Check internet/availability or pick another model.")

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)


In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


In [ ]:
max_length = 1024
pad_id = tokenizer.pad_token_id

def build_prompt(instruction, inp):
    return f"### Instruction:\n{instruction}\n\n### Input:\n{inp}\n\n### Response:\n"

def tokenize_and_make_labels(example):
    # Build prompt and response separately to create labels that ignore prompt
    prompt = build_prompt(example["instruction"], example["input"])
    response = example["output"]
    # Encode without special tokens to control concatenation
    p_ids = tokenizer.encode(prompt, add_special_tokens=False)
    r_ids = tokenizer.encode(response, add_special_tokens=False)
    input_ids = (p_ids + r_ids)[:max_length]
    # Create labels: -100 for prompt tokens, response ids for response region
    labels = [-100] * len(p_ids) + r_ids
    labels = labels[:max_length]
    # Pad if needed
    if len(input_ids) < max_length:
        pad_len = max_length - len(input_ids)
        input_ids = input_ids + [pad_id] * pad_len
        labels = labels + [-100] * pad_len
    attention_mask = [1 if id != pad_id else 0 for id in input_ids]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

# Map across dataset (batched=False for clarity; adjust if you hit speed issues)
tokenized = ds.map(lambda ex: tokenize_and_make_labels(ex), remove_columns=ds.column_names)


###Training arguements###

In [ ]:
from transformers import TrainingArguments, Trainer

def collate_fn(batch):
    import torch
    input_ids = torch.tensor([b["input_ids"] for b in batch], dtype=torch.long)
    attention_mask = torch.tensor([b["attention_mask"] for b in batch], dtype=torch.long)
    labels = torch.tensor([b["labels"] for b in batch], dtype=torch.long)
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


In [ ]:
training_args = TrainingArguments(
    output_dir="./mly_qlora_out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    remove_unused_columns=False,
    optim="paged_adamw_32bit"
)

###Trainer###

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=collate_fn,
    tokenizer=tokenizer,
)

print("Starting training... (this will use QLoRA adapters and is GPU heavy)")
trainer.train()

In [ ]:
adapter_dir = "./mly_finetuned_adapter"
model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(adapter_dir)
print("Saved adapter + tokenizer to", adapter_dir)


###Evaluation###

In [ ]:
from transformers import pipeline

def generate_summary(chapter_text, max_new_tokens=250):
    prompt = f"### Instruction:\nSummarize the following chapter from Machine Learning Yearning in detail:\n\n### Input:\n{chapter_text}\n\n### Response:\n"
    gen = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto", torch_dtype=torch.float16)
    out = gen(prompt, max_new_tokens=max_new_tokens, do_sample=False)
    text = out[0]["generated_text"]
    return text.split("### Response:")[-1].strip()

In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge_score

In [ ]:
from transformers import pipeline
import evaluate
import pandas as pd
import time


summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

# Load ROUGE metric
rouge = evaluate.load("rouge")

# Example small manual evaluation set
data = pd.DataFrame({
    "text": [
        "Deep learning is a subset of machine learning that uses neural networks.",
        "Reinforcement learning trains an agent to take actions to maximize rewards."
    ],
    "reference_summary": [
        "Deep learning is a type of ML using neural networks.",
        "Reinforcement learning optimizes actions for rewards."
    ]
})

pred_summaries = []
start_time = time.time()

for doc in data["text"]:
    summary = summarizer(
        doc,
        max_length=50,
        min_length=10,
        do_sample=False,
        truncation=True
    )[0]["summary_text"]
    pred_summaries.append(summary)

elapsed = time.time() - start_time

# Compute ROUGE
results = rouge.compute(
    predictions=pred_summaries,
    references=data["reference_summary"]
)

# Display results
print("📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")

print(f"\n⏱ Processed {len(data)} samples in {elapsed:.2f} seconds (~{elapsed/len(data):.2f} sec/sample)")

# 📝 Note
print("\nNOTE: The above scores are based on a very small manual test set for demonstration. "
      "If the official evaluation dataset lacks valid summaries, ROUGE will be close to zero. "
      "The purpose here is to verify that the fine-tuned model pipeline runs end-to-end.")


In [ ]:
#Show one example summary from the fine-tuned model
sample_text = data["text"].iloc[0]
reference_summary = data["reference_summary"].iloc[0]

# Generate summary
generated_summary = summarizer(
    sample_text,
    max_length=50,
    min_length=10,
    do_sample=False,
    truncation=True
)[0]["summary_text"]

print("=== Example Summary ===")
print("\n📜 Original Text:\n", sample_text)
print("\n✅ Reference Summary:\n", reference_summary)
print("\n🤖 Generated Summary:\n", generated_summary)
